In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from MUA_helper import (make_NP_array_from_linear_data, cleanAxes, placeAxesOnGrid,filter_detect,butter_bandpass,
                        butter_bandpass_filter,basic_peak_detector,get_prepare_PSTHs,plot_save,plot_save_single)

from scipy import stats
import pickle 
import pandas as pd
import scipy.io as sio
import os

pom 10,17 // 377
vpl 9,16 // 384

In [ ]:
test = np.load(r"Y:\Data\Neuropixel_M1\Kilosort single units\2019-08-28_14-02-10\spike_times.npy")

In [ ]:
((test[-1]/30000) - (test[0]/30000))/60

In [ ]:
structure = 'M1'
exp_name = 'movement'

working_dir = r"Y:\Data\Neuropixel_M1\2019-08-30_17-28-19"


stim_data_path = os.path.join(working_dir,'MUA-Analysis','timestamps','2019-08-30_17-28-19_trigger_move.npy')

binsize = 0.100
samplingrate = 30000
sweeplength = 5
ch_bin = 4
pre = 0
chs = 384
trials = 100
thresh = -5
duration=0.5
baselines = []
stimuli = []

baselinestart = 3
baselineend = 5
stimulusstart = 0
stimulusend = 2

save = False


colors = plt.cm.coolwarm(np.linspace(0,1,8))


In [ ]:
data_corrected_list = []
baselines = []
stimuli = []


timestamps = np.load(stim_data_path)
    
    
for i in range(len(timestamps))[:]:
    
    color = colors[i]
    folder = 'timestamps_{}'.format(i)
    folder_path  = working_dir + '/MUA-Analysis/' + exp_name + '/'+str(thresh) + '/timestamps_{}/'.format(i)

    files = os.listdir(folder_path)
    files = files[:]
    #with open(stim_data_path, 'rb') as f:
    #   x = pickle.load(f)

    #feedback = x["Feedback"][stimulus]

    all_trials = {}
    for ch in range(chs):
        all_trials[str(ch)]= np.array([])
    print('dict created....')
    for trial in range(len(files)):
        test0 = np.load(os.path.join(folder_path,files[trial]))
        for ch in range(test0.shape[0]):
            all_trials[str(ch)] = np.sort(np.hstack((all_trials[str(ch)],test0[ch])))

    print('dict filled....')


    binwindow = binsize*samplingrate
    data = np.zeros([chs,int(sweeplength/binsize)-1])   
    bins = np.arange(0,sweeplength*samplingrate,binwindow)
    data_df = pd.DataFrame([])

    for x in range(chs):

        values,edges = np.histogram(all_trials[str(x)],bins)
        data[x,:] = values/binsize/trials

    data_shape = np.reshape(data, (ch_bin,-1),order='F')
    data_db = np.mean(data_shape,axis =0)
    data_new = np.reshape(data_db,(int(chs/ch_bin),-1),order='F')
    
    
    for t in range(int(chs/ch_bin)):
        data_df[str(t)] = data_new[t,:]
    
    baseline_df = data_df.iloc[int(baselinestart/binsize):int(baselineend/binsize)]
    baseline_stats = baseline_df.describe().T
    baseline_stats['var'] = baseline_df.var()
    data_corrected = pd.DataFrame([])
    baselines.append(baseline_stats)
    

    stimulus_df = data_df.iloc[int(stimulusstart/binsize):int(stimulusend/binsize)]
    stimulus_stats = stimulus_df.describe().T
    stimuli.append(stimulus_stats)

    for ii in range(int(chs/ch_bin)):

        data_corrected[str(ii)] = (data_df[str(ii)] - baseline_stats['mean'].iloc[ii])#/baseline_stats['std'].iloc[ii]
        
    data_corrected_list.append(data_corrected)
    path =os.path.join(working_dir ,'MUA-Analysis', exp_name,str(thresh),'plots','base_corrected',folder)    
    savepath= os.path.join(path,structure+'.png')
    if not os.path.isdir(path):
        os.makedirs(path)

    ticks = np.arange(0,int(data_corrected.shape[0]),1/binsize)
    time = np.arange(-pre,sweeplength-pre,binsize)



    fig = plt.figure(figsize=(12,10))

    ax1 = placeAxesOnGrid(fig)


    sns.heatmap(data_corrected.T,vmax=15,vmin =-5,ax=ax1,cbar_kws={'label': 'firing rate'})
    #sns.heatmap(data_corrected.T,vmax=20,ax=ax1,cbar_kws={'label': 'z-score'})
    plt.ylim(0,int(chs/ch_bin))
    plt.show()
    
    plt.ylim(0,int(chs/ch_bin))
    ax1.set_xticks(ticks)
    ax1.set_xticklabels(time[ticks.astype(int)].round().astype(int),rotation = 0)
    ax1.set_xlabel('time',fontsize = 15)
    ax1.axvline(pre/binsize,color = 'white',lw = 0.5)
    ax1.axvline((pre+duration)/binsize,color = 'white',lw = 0.5)
    
    ax1.set_ylim(0,data_corrected.shape[1])
    ax1.figure.axes[-1].yaxis.label.set_size(15)

    if save:
        plt.savefig(str(savepath))
    plt.show()

    

stimuli_all = pd.DataFrame([])
baselines_all = pd.DataFrame([])
        
for tt in range(len(baselines)):
    baselines[tt]['exp'] = tt
    stimuli[tt]['exp'] = tt
    
stimuli_all = pd.concat(stimuli)
baselines_all = pd.concat(baselines)

savepath1 = os.path.join(os.path.dirname(path),structure+'_stim.csv')
if save:
    stimuli_all.to_csv(savepath1)

savepath2 = os.path.join(os.path.dirname(path),structure+'_base.csv')
if save:
    baselines_all.to_csv(savepath2)

In [ ]:
stimuli_all= pd.read_csv(savepath1,index_col=0)
baselines_all = pd.read_csv(savepath2,index_col=0)

In [ ]:
#z-score data
stimuli_all['fr corrected'] = (stimuli_all['mean'] - baselines_all['mean']).values#/baselines_all['std']

thr = 1.5
stimuli_all_resp_tmp = stimuli_all[(abs(stimuli_all['fr corrected'])>thr)&(stimuli_all['exp']==0)]
stimuli_all_resp = stimuli_all.loc[stimuli_all_resp_tmp.index]
amps = pd.read_pickle(metaData_path)
amps = np.unique(amps['Stimulus Temp']).astype(int)

In [ ]:
os.path.dirname(path)

In [ ]:
% matplotlib inline
for i in range(areas.shape[0]):
    fig = plt.figure(figsize=(5,6))
    ax = placeAxesOnGrid(fig)
    cleanAxes(ax,leftLabels=True)
    
    data_tmp = stimuli_all[(stimuli_all.index.values.astype(int) > int(areas[i][0]))&
                                             (stimuli_all.index.values.astype(int) < int(areas[i][-1]))]
    

    sns.stripplot(x='exp',y='fr corrected',data=data_tmp,palette = (sns.color_palette("coolwarm", n_colors=len(colors))),jitter = True,ax=ax)
    ax.plot(np.unique(data_tmp['exp']),np.zeros(np.unique(data_tmp['exp']).shape[0]),color='grey',ls='--')
    ax.set_ylim(-5,15)
    ax
    ax.set_xticklabels(amps-32,fontsize=12)
    #ax.set_yticks(np.arange(-5,10,1))
    #ax.set_yticklabels(np.arange(-5,10,1),fontsize=12)
    ax.set_xlabel('Stimulus Temp', fontsize = 15)
    ax.set_ylabel('firing rate', fontsize = 15)
    areas_stats = data_tmp.groupby('exp').median()
    
    window = 0.3
    
    for ii in range(np.unique(data_tmp['exp']).shape[0]):
        amp_tmp = np.unique(data_tmp['exp'])[ii]
        ax.plot([amp_tmp-window,amp_tmp+window],[areas_stats['fr corrected'][ii],areas_stats['fr corrected'][ii]],color='black')    
    
    savepath_tmp = os.path.join(os.path.dirname(path),structure+'_chs_{}_{}.png'.format(areas[i][0],areas[i][-1]))
    plt.savefig(savepath_tmp)
    plt.show()

In [ ]:
% matplotlib inline

data_df_corr = data_df.corr(method = 'pearson')

sns.heatmap(data_df_corr,vmin=-1,vmax=1)
plt.show()

In [ ]:
% matplotlib inline 

start = 9
end = 13

window = np.arange(start*1000,end*1000)
feedback_all =x["Feedback_Force"]

for t in range(1):
    
    print('new amplitude')
    count = 0
    
    feedback = feedback_all[t][:,0,:]
    
    for i in range(25):
        smooth_feedback = Gaussian_smooth(feedback[:,i],250)
        plt.plot(smooth_feedback,lw = 0.8,alpha = 0.5)
        #plt.xlim(8000,12000)
        plt.ylim(0,10)
        peaks = basic_peak_detector(feedback[:,i], thresh=-1, orientation='positive', verbose=False)
        peaks = peaks[(peaks > window[0]) & (peaks < window [-1])]
        if peaks.size > 0:
            count +=1
            
            print('{} trials detected with paw movement during temp stim'.format(count))
        plt.show()
        

In [ ]:
baselinestart = 7
baselineend = 9

baseline_df = data_areas.iloc[int(baselinestart/binsize):int(baselineend/binsize)]
baseline_stats = baseline_df.describe().T
data_corrected = pd.DataFrame([])
baselines.append(baseline_stats)

for i in range(3):

    data_areas_corrected[str(i)] = (data_areas[str(i)] - baseline_stats['mean'].iloc[i])/baseline_stats['std'].iloc[i]

In [ ]:
for i in range(3):
    plt.plot(data_areas_corrected.iloc[:,i])
plt.xlim(35,45)
plt.show()